# Tests

In [ ]:
;; At the command line, create a new project called 'inventory'

"""
!lein new inventory
"""


### Spotting Bugs with clojure.test

In [ ]:
;; A new book inventory will look like the following

[{:title "2001" :author "Clarke" :copies 21}
 {:title "Emma" :author "Austen" :copies 10}
 {:title "Misery" :author "King" :copies 101}]


In [ ]:
;; In inventory/src/inventory/core.clj, save the first 2 functions 

(ns inventory.core)

(defn find-by-title
    "Search for a book by title,
    where title is a string and books is a collection
    of book maps, each of which must have a :title entry"
    [title books]
    (some #(when (= (:title %) title) %) books))

(defn number-of-copies-of
    "Return the number of copies in inventory of the
    given title, where title is a string and books is a collection
    of book maps each of which must have a :title entry"
    [title books]
    (:copies (find-by-title title books)))


In [ ]:
;; In inventory/test/inventory/core_test.clj, 
;; save the the first test, as follows

(ns inventory.core-test
    (:require [clojure.test :refer :all])
    (:require [inventory.core :as i]))

(def books
    [{:title "2001" :author "Clarke" :copies 21}
     {:title "Emma" :author "Austen" :copies 10}
     {:title "Misery" :author "King" :copies 101}])

(deftest test-finding-books
    (is (not (nil? (i/find-by-title "Emma" books)))))



In [ ]:
;; In the root directory 'inventory' of the project, 
;; start a REPL and execute the first test

"""
!lein repl
"""

(require '[inventory.core-test :as ct])

(ct/test-finding-books) ; The test should succeed


In [ ]:
;; You can also use more than one expression per test, as follows

(deftest test-finding-books-better
    (is (not (nil? (i/find-by-title "Emma" books))))
    (is (nil? (i/find-by-title "XYZZY" books))))


In [ ]:
;; You can even organize your tests into subtests with 'testing', as follows

(deftest test-basic-inventory
    (testing "Finding books"
        (is (not (nil? (i/find-by-title "Emma" books))))
        (is (nil? (i/find-by-title "XYZZY" books))))
    (testing "Copies in inventory"
        (is (= 10 (i/number-of-copies-of "Emma" books)))))


In [ ]:
;; In the REPL, execute the 2 new tests, as follows

(require '[inventory.core-test :as ct])

(ct/test-finding-books-better) ; Both tests should succeed
(ct/test-basic-inventory) 


### Testing Namespaces and Projects

In [ ]:
;; Since running tests manually doesn't scale, you can use the 'run-tests' function
;; to rull all tests saved at inventory/test/inventory, as follows

(require '[clojure.test])

(clojure.test/run-tests) ; Running tests without args
(clojure.test/run-tests *ns*)  ; Running tests in the current namespace
(ns inventory.core-test)
(clojure.test/run-tests 'inventory.core-test) ; Running tests in the 'inventory.core-test' namespace


In [ ]:
;; The following leiningen command can also be used to run all tests

"""
lein test
"""


### Property-based testing

In [ ]:
;; You can generate random test data with the 'generators' library, as follows

(require '[clojure.test.check.generators :as gen])

(gen/sample gen/string-alphanumeric)

In [ ]:
;; With the last library, you can generate test date 
;; for  the 'inventory' app, as follows

(def title-gen gen/string-alphanumeric)

(def author-gen gen/string-alphanumeric)

(def copies-gen gen/pos-int)


In [ ]:
;; To generate nonempty titles and authors, and nonzero number of copies,
;; use the 'such-that' function and a predicate, as follows

(def title-gen (gen/such-that not-empty gen/string-alphanumeric))

(def author-gen (gen/such-that not-empty gen/string-alphanumeric))

(def copies-gen (gen/such-that (complement zero?) gen/pos-int))

In [ ]:
;; With the last functions, we can generate maps of book data
;; with the 'hash-map' function, as follows

(def book-gen
    (gen/hash-map :title title-gen :author author-gen :copies copies-gen))



In [ ]:
;;And we can generate random data for the whole inventory, as follows

(def inventory-gen 
    (gen/not-empty (gen/vector book-gen)))


In [ ]:
;; We can also generate one book along each inventory with 
;; the let version of the generator 'gen/let', as follows

(def inventory-and-book-gen
    (gen/let [inventory inventory-gen
              book (gen/elements inventory)]
    {:inventory inventory :book book}))



### Checking Properties

In [ ]:
;; As a simple example, we can state that each positive integer is smaller 
;; than the next positive integer with 'for-all', as follows

(require '[clojure.test.check.properties :as prop])

(prop/for-all [i gen/pos-int]
    (< i (inc i)))


In [ ]:
;; But we should check the property with a finite number of cases.
;; We can do that with 'quick-check', as follows

(require '[clojure.test.check :as tc])

(tc/quick-check 50
    (prop/for-all [i gen/pos-int]
        (< i (inc i))))

In [ ]:
;; From the last examples, we can create the tests for the 'inventory' project, as follows

(defn my-property-test []
    (tc/quick-check 50
        (prop/for-all [i-and-b inventory-and-book-gen]
            (=  (i/find-by-title (-> i-and-b :book :title) (:inventory i-and-b)) (:book i-and-b)))))
